In [6]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "github_repos" dataset
dataset_ref = client.dataset("github_repos", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "sample_commits" table
table_ref = dataset_ref.table("sample_commits")

# API request - fetch the table
sample_commits_table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(sample_commits_table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,commit,tree,parent,author,committer,subject,message,trailer,difference,difference_truncated,repo_name,encoding
0,afdba32e2a9ea729a9f9f280dbf6c718773c7ded,d77cca8a096e5320f3194d4a6ca1b4fef2dc9b99,[d65e55d4999b394e37ffe12543ecd2a17b7c44fc],"{'name': 'Jason Gunthorpe', 'email': 'a99b91d7...","{'name': 'Peter Huewe', 'email': '014f16385c5a...",tpm: Pull everything related to /dev/tpmX into...,tpm: Pull everything related to /dev/tpmX into...,"[{'key': 'Signed-off-by', 'value': 'Jason Gunt...","[{'old_mode': 33188.0, 'new_mode': 33188, 'old...",None,torvalds/linux,None
1,eb846d9f147455e4e5e1863bfb5e31974bb69b7c,443efbb146c7824508be817923bab04c2185810e,[3af6b35261182ff185db1f0fd271254147e2663e],"{'name': 'Hannes Reinecke', 'email': 'b0d1e9e4...","{'name': 'Christoph Hellwig', 'email': '923f77...",scsi: rename SERVICE_ACTION_IN to SERVICE_ACTI...,scsi: rename SERVICE_ACTION_IN to SERVICE_ACTI...,"[{'key': 'Signed-off-by', 'value': 'Hannes Rei...","[{'old_mode': 33188.0, 'new_mode': 33188, 'old...",None,torvalds/linux,None
2,f8798ccbefc0e4ef7438c080b7ba0410738c8cfa,9133440693c02314f1f6f95629b3594ce24ad0f8,[261e767628bb5971b9032439818237cc8511ea94],"{'name': 'Yong Zhang', 'email': '34add0fe16a1f...","{'name': 'Florian Tobias Schandinat', 'email':...",video: irq: Remove IRQF_DISABLED,video: irq: Remove IRQF_DISABLED\n\nSince comm...,"[{'key': 'Signed-off-by', 'value': 'Yong Zhang...","[{'old_mode': 33188.0, 'new_mode': 33188, 'old...",None,torvalds/linux,None
3,b83ae6d421435c6204150300f1c25bfbd39cd62b,99c6b661ab7de05c2fd49aa62624d2d6bf8abc69,[de1414a654e66b81b5348dbc5259ecf2fb61655e],"{'name': 'Christoph Hellwig', 'email': '923f77...","{'name': 'Jens Axboe', 'email': 'cd8c6775e60d6...",fs: remove mapping->backing_dev_info,fs: remove mapping->backing_dev_info\n\nNow th...,"[{'key': 'Signed-off-by', 'value': 'Christoph ...","[{'old_mode': 33188.0, 'new_mode': 33188, 'old...",None,torvalds/linux,None
4,aaabee8b7686dfe49f10289cb4b7a817b99e5dd9,7ccc6cf829a93d46daf484164a5466c91eca2efa,"[795e9364215dc98b1dea888ebae22383ecbbb92a, 2f2...","{'name': 'Luciano Coelho', 'email': 'd1ef58086...","{'name': 'Luciano Coelho', 'email': 'd1ef58086...",Merge branch 'wl12xx-next' into for-linville,Merge branch 'wl12xx-next' into for-linville\n...,"[{'key': 'Conflicts', 'value': '', 'email': No...","[{'old_mode': 33188.0, 'new_mode': 33188, 'old...",None,torvalds/linux,None


Run the next code cell to print the table schema. 

In [9]:
# Print information on all the columns in the table
sample_commits_table.schema

[SchemaField('commit', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('tree', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('parent', 'STRING', 'REPEATED', None, (), None),
 SchemaField('author', 'RECORD', 'NULLABLE', None, (SchemaField('name', 'STRING', 'NULLABLE', None, (), None), SchemaField('email', 'STRING', 'NULLABLE', None, (), None), SchemaField('time_sec', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('tz_offset', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('date', 'TIMESTAMP', 'NULLABLE', None, (), None)), None),
 SchemaField('committer', 'RECORD', 'NULLABLE', None, (SchemaField('name', 'STRING', 'NULLABLE', None, (), None), SchemaField('email', 'STRING', 'NULLABLE', None, (), None), SchemaField('time_sec', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('tz_offset', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('date', 'TIMESTAMP', 'NULLABLE', None, (), None)), None),
 SchemaField('subject', 'STRING', 'NULLABLE', None, (), None),
 SchemaFie

In [10]:
# commiter: tenemos una anidacion, tenemos que usar el .
# 

# Write a query to find the answer
# old query
# max_commits_query = """
# SELECT committer.name AS committer_name, COUNT(1)
# FROM `bigquery-public-data.github_repos.sample_commits`
# WHERE committer.date = 2016

#                     """
# new_query
max_commits_query = """
SELECT 
    committer.name AS committer_name, 
    COUNT(*) AS num_commits
FROM 
    `bigquery-public-data.github_repos.sample_commits`
WHERE 
    EXTRACT(YEAR FROM committer.date) = 2016
GROUP BY 
    committer_name
ORDER BY 
    num_commits DESC
"""



,committer_name,num_commits
0,Greg Kroah-Hartman,3545
1,David S. Miller,3120
2,TensorFlower Gardener,2449
3,Linus Torvalds,2424
4,Benjamin Pasero,1127


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [13]:
# Construct a reference to the "languages" table
table_ref = dataset_ref.table("languages")

# API request - fetch the table
languages_table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(languages_table, max_results=5).to_dataframe()

,repo_name,language
0,lemi136/puntovent,"[{'name': 'C', 'bytes': 80}]"
1,taxigps/nctool,"[{'name': 'C', 'bytes': 4461}]"
2,ahy1/strbuf,"[{'name': 'C', 'bytes': 5573}]"
3,nleiten/mod_rpaf-ng,"[{'name': 'C', 'bytes': 30330}]"
4,kmcallister/alameda,"[{'name': 'C', 'bytes': 17077}]"


In [14]:
# Print information on all the columns in the table
languages_table.schema

[SchemaField('repo_name', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('language', 'RECORD', 'REPEATED', None, (SchemaField('name', 'STRING', 'NULLABLE', None, (), None), SchemaField('bytes', 'INTEGER', 'NULLABLE', None, (), None)), None)]

Assume for the moment that you have access to a table called `sample_languages` that contains only a very small subset of the rows from the `languages` table: in fact, it contains only three rows!  This table is depicted in the image below.

![](https://storage.googleapis.com/kaggle-media/learn/images/qAb5lZ2.png)

How many rows are in the table returned by the query below?

![](https://storage.googleapis.com/kaggle-media/learn/images/Q5qYAtz.png)

Fill in your answer in the next code cell.

In [15]:
# Fill in the blank
num_rows = 6

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [29]:
# Write a query to find the answer # me olvide de la coma en el from --> 
# pop_lang_query = """
# SELECT lang.name AS language_name, COUNT(*) AS num_repos
# FROM `bigquery-public-data.github_repos.sample_languages`, # llame mal a la tabla 
#     UNNEST(language) AS lang
# GROUP BY lang.name
# ORDER BY num_repos DESC
# """




pop_lang_query = """
                 SELECT l.name as language_name, COUNT(*) as num_repos
                 FROM `bigquery-public-data.github_repos.languages`,
                     UNNEST(language) AS l
                 GROUP BY language_name
                 ORDER BY num_repos DESC
                 """

,language_name,num_repos
0,JavaScript,1099966
1,CSS,807826
2,HTML,777433
3,Shell,640886
4,Python,550905


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [34]:
# Your code here  
# all_langs_query = """
#                  SELECT l.name as name, SUM(l.bytes) as bytes
#                  FROM `bigquery-public-data.github_repos.languages`,
#                      UNNEST(language) AS l
#                  GROUP BY name
#                  WHERE repo_name LIKE 'polyrabbit/polyglot'
#                  ORDER BY bytes DESC
#                  """

all_langs_query = """
                  SELECT l.name, l.bytes
                  FROM `bigquery-public-data.github_repos.languages`,
                      UNNEST(language) as l
                  WHERE repo_name = 'polyrabbit/polyglot'
                  ORDER BY l.bytes DESC
                  """

,name,bytes
0,Lasso,834726
1,C,819142
2,Mercury,709952
3,Objective-C,495392
4,Game Maker Language,298131


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>